In [1]:
import review_RTP as RTP
import numpy as np
import geopandas as gpd
import os
import pandas as pd

In [2]:
targetLayers = RTP.targetLayers()

In [3]:
inpath = r'T:\MPO\RTP\FY16 2040 Update\Data\RTP_2040_Data.gdb'
path = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data'

In [7]:
tablePatterns = np.array(['Auto Constrained', 'Auto Illustrative', 'Bike Constrained', 'Bike Illustrative'])
layerPatterns = np.array(['Constrained_Roadway', 'Illustrative_Roadway', 'Constrained_BikePed', 'Illustrative_BikePed'])

In [8]:
for layer in targetLayers:
    print(layer)
    gdf = gpd.read_file(inpath, layer=layer)
    l = layer.split('_')
    layerPattern = l[0] + '_' + l[1] 
    i=np.min(np.where(layerPatterns == layerPattern))
    tablerPattern = tablePatterns[i]
    res=RTP.getIDs(Tablepattern=tablerPattern,
           Layerpattern=layerPattern)
    commonIDs=RTP.matchID(res[0], res[1])[2]
    newgdf = gdf[gdf.RTP_ID.isin(commonIDs)]
    newgdf.to_file(os.path.join(path, layer+'.shp'))

Constrained_Roadway_lines


<ipython-input-8-e935b767af78>:12: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  newgdf.to_file(os.path.join(path, layer+'.shp'))


Constrained_Roadway_points
Illustrative_Roadway_lines
Illustrative_Roadway_points
Constrained_BikePed
Constrained_BikePed_points
Illustrative_BikePed


In [2]:
df = RTP.modifyRTP(RTP.combineTables())

In [3]:
df.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,Category
0,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,7.666397e+07,8.662156e+07,0.0,27,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,5.264874e+07,5.948708e+07,0.0,30,New Arterial Link or Interchange
2,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new arterial bridge with up to 4 lan...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,1.092461e+08,1.234357e+08,0.95,512,New Arterial Link or Interchange
3,Delta/ Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,2.259772e+07,2.553285e+07,0.25,638,Added Freeway Lanes or Major Interchange Impro...
4,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Randy Pape Belltine Highway and p...,"ODOT, Eugene",28100000.0,2030-2034,4.308515e+07,4.868131e+07,1.1,312,Added Freeway Lanes or Major Interchange Impro...


In [4]:
df.shape

(318, 11)

In [5]:
df45 = RTP.modifyRTP(RTP.combineTables(year=2045))

In [6]:
df45.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,Category
0,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new 2-lane arterial bridge over the ...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,NaN,NaN,0.95,512,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,NaN,NaN,0.0,27,New Arterial Link or Interchange
2,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,NaN,NaN,0.0,30,New Arterial Link or Interchange
3,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",28100000.0,2030-2034,NaN,NaN,1.1,312,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,NaN,NaN,0.25,638,Added Freeway Lanes or Major Interchange Impro...


In [7]:
len([N for N in list(df.RTP.unique()) if N in list(df45.RTP.unique())])

224

In [8]:
len([N for N in list(df.RTP.unique()) if N not in list(df45.RTP.unique())])

30

In [9]:
[N for N in list(df45.RTP.unique()) if N not in list(df.RTP.unique())]

[]

In [6]:
df45.shape

(273, 11)

In [11]:
data = RTP.checkDiff()

In [12]:
data.columns

Index(['Name40', 'GeographicLimits40', 'Description40',
       'PrimaryJurisdiction40', 'EstimatedCost40',
       'EstimatedYearofConstruction40', 'YearofConstructionCostMin40',
       'YearofConstructionCostMax40', 'Length40', 'RTP40', 'Category40',
       'Name45', 'GeographicLimits45', 'Description45',
       'PrimaryJurisdiction45', 'EstimatedCost45',
       'EstimatedYearofConstruction45', 'YearofConstructionCostMin45',
       'YearofConstructionCostMax45', 'Length45', 'RTP45', 'Category45',
       'NameDiff', 'GeographicLimitsDiff', 'DescriptionDiff',
       'PrimaryJurisdictionDiff', 'EstimatedCostDiff',
       'EstimatedYearofConstructionDiff', 'YearofConstructionCostMinDiff',
       'YearofConstructionCostMaxDiff', 'LengthDiff', 'RTPDiff',
       'CategoryDiff'],
      dtype='object')

In [13]:
data['GeographicLimitsDiff'].value_counts()

0    139
Name: GeographicLimitsDiff, dtype: int64

In [14]:
data['CategoryDiff'].value_counts()

0    139
Name: CategoryDiff, dtype: int64

In [15]:
data['LengthDiff'].value_counts()

0.0    138
Name: LengthDiff, dtype: int64

In [17]:
(df45.shape[0] - data.shape[0])/df45.shape[0]

0.4908424908424908

In [7]:
dfex = RTP.modifyRTP(RTP.combineTables(excludeTransit = True))

In [10]:
dfex.shape

(298, 13)

In [11]:
df45ex = RTP.modifyRTP(RTP.combineTables(year=2045, excludeTransit = True))

In [9]:
df45ex.shape

(254, 12)

In [2]:
dataex = RTP.checkDiff(excludeTransit = True, by='RTP')

In [3]:
dataex.columns

Index(['Name40', 'GeographicLimits40', 'Description40',
       'PrimaryJurisdiction40', 'EstimatedYearofConstruction40', 'Length40',
       'RTP', 'JurisdictionalProject#40', 'Comments40', 'Category40', 'Name45',
       'GeographicLimits45', 'Description45', 'PrimaryJurisdiction45',
       'EstimatedYearofConstruction45', 'Length45', 'JurisdictionalProject#45',
       'Category45', 'NameDiff', 'GeographicLimitsDiff', 'DescriptionDiff',
       'PrimaryJurisdictionDiff', 'EstimatedYearofConstructionDiff',
       'LengthDiff', 'JurisdictionalProject#Diff', 'CategoryDiff'],
      dtype='object')

In [4]:
dataex.shape

(176, 26)

In [12]:
df45ex.shape[0] - dataex.shape[0]

78

In [5]:
dataex.head()

,Name40,GeographicLimits40,Description40,PrimaryJurisdiction40,EstimatedYearofConstruction40,Length40,RTP,JurisdictionalProject#40,Comments40,Category40,...,JurisdictionalProject#45,Category45,NameDiff,GeographicLimitsDiff,DescriptionDiff,PrimaryJurisdictionDiff,EstimatedYearofConstructionDiff,LengthDiff,JurisdictionalProject#Diff,CategoryDiff
0,Delta/ Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,2020-2024,0.25,638,NaN,NaN,Added Freeway Lanes or Major Interchange Impro...,...,ETSP MM-3 (part),Added Freeway Lanes or Major Interchange Impro...,999,NaN,999,0,0,0.0,999.0,0
1,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Randy Pape Belltine Highway and p...,"ODOT, Eugene",2030-2034,1.1,312,MM-20,Updated - Rob,Added Freeway Lanes or Major Interchange Impro...,...,ETSP MM-20,Added Freeway Lanes or Major Interchange Impro...,0,0.0,999,0,0,0.0,999.0,0
2,Main Street,@ 48th Street,Traffic control improvements,Springfield,2020-2024,0.0,69,NaN,change cost from $290k to $300k,Arterial Capacity Improvements,...,NaN,Arterial Capacity Improvements,0,999.0,0,0,999,0.0,NaN,0
3,Main Street,@ Mountaingate Drive,Traffic control improvements - signal,Springfield,2020-2024,0.0,75,R-48,change cost from $290k to $900k,Arterial Capacity Improvements,...,R-48,Arterial Capacity Improvements,0,999.0,0,0,999,0.0,0.0,0
4,42nd Street,@ Marcola Road,Roundabout,Springfield,2025-2029,0.0,712,R-32,change to roundabout project and change cost f...,Arterial Capacity Improvements,...,R-32,Arterial Capacity Improvements,0,999.0,0,0,999,0.0,0.0,0


In [7]:
dataex['NameDiff'].value_counts()

0      158
999     18
Name: NameDiff, dtype: int64

In [13]:
dataex['GeographicLimitsDiff'].value_counts()

0.0      140
999.0     34
Name: GeographicLimitsDiff, dtype: int64

In [15]:
dataex['LengthDiff'].value_counts()

 0.00    174
 0.07      1
-0.70      1
Name: LengthDiff, dtype: int64

In [18]:
dataex['CategoryDiff'].value_counts()

0    176
Name: CategoryDiff, dtype: int64

In [9]:
dataex[dataex.NameDiff == 999][['Name40', 'Description40', 'Name45', 'Description45', 'RTP']]

,Name40,Description40,Name45,Description45,RTP
0,Delta/ Beltline Interchange,Interim/safety improvements; replace/revise ex...,Delta/Beltline Interchange,Interim/safety improvements; replace/revise ex...,638
33,Glacier Drive,Construct new collector with 2-lane cross-section,New Collector,Construct new collector with 2-lane cross-sect...,22
56,Main St. and 52nd St./Hwy 126 Int.,Interchange Plans,OR126E (Expressway) at 52nd St and Main St,Interchange Area Management Plans,96
57,Eugene-Springfield Hwy.,Facility Plan,OR126 Expressway,Facility Plan,835
61,OR 126,Study a new crossing of OR 126 between 5th Str...,Main Street (OR126B),Study a new crossing of OR 126 between 5th Str...,823
70,Main Street,Access plan study,Main Street (OR126B),Facility Plan,917
75,South 14th Street,Extend South 14th Street south of the Union Pa...,South 14th Street Extension,Extend South 14th Street south of the Union Pa...,825
76,New Collector,Extend South B Street with a 3-lane cross-sect...,New Collector - South B Street,Extend South B Street with a 3-lane cross-sect...,913
86,Eugene-Springfield Highway (SR-126),Interchange improvements,Eugene-Springfield Highway (OR-126E) at Pionee...,Interchange improvements,727
87,Eugene-Springfield Highway (SR-126),Widen to 6 lanes,Eugene-Springfield Highway (OR-126E),Widen to 6 lanes,728


In [14]:
dataex[dataex.GeographicLimitsDiff == 999][['GeographicLimits40', 'Description40', 'GeographicLimits45', 'Description45', 'RTP']]

,GeographicLimits40,Description40,GeographicLimits45,Description45,RTP
2,@ 48th Street,Traffic control improvements,at 48th Street,Traffic control improvements,69
3,@ Mountaingate Drive,Traffic control improvements - signal,at Mountaingate Drive,Traffic control improvements - signal,75
4,@ Marcola Road,Roundabout,at Marcola Road,Roundabout,712
5,@ Pheasant Boulevard,Traffic control improvements,at Pheasant Blvd,Traffic control improvements,744
6,@ Harlow Road,Traffic control improvements,at Harlow Road,Traffic control improvements,785
21,Bob Straub Parkway - Mountaingate Drive,Construct new 3-lane collector,Bob Straub Parkway to Mountaingate Drive and F...,Construct a new collector with a three-lane cr...,81
25,31st Street to 33rd Street,Extend existing street as 2-lane collector,31st Street to 35th Street,Construct Yolanda Avenue from 31st Street to 3...,783
28,"Game Farm Road East, to International Way",Construct new 3- lane collector,Game Farm Road East to International Way,Construct new 3- lane collector with sidewalks...,707
33,48th Street/Holly to South 55th Street,Construct new collector with 2-lane cross-section,Holly Street - South 48th Street to South 57th...,Construct new collector with 2-lane cross-sect...,22
42,Warren to UGB,Upgrade to urban facility,Warren St to Eugene UGB,Construct to eugene's minor arterial standards...,343


In [17]:
dataex[dataex.LengthDiff != 0][['Length40', 'Description40', 'Length45', 'Description45', 'RTP']]

,Length40,Description40,Length45,Description45,RTP
13,0.91,Add center turn lane.,0.21,Add center turn lane on Martin Luther King Jr....,602
14,0.07,Widen Barger Drive to provide a second through...,0.14,Widen Barger Drive to provide a second through...,497


In [5]:
dfex.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,JurisdictionalProject#,Comments,Category
0,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,7.666397e+07,8.662156e+07,0.0,27,NaN,NaN,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,5.264874e+07,5.948708e+07,0.0,30,NaN,NaN,New Arterial Link or Interchange
2,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new arterial bridge with up to 4 lan...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,1.092461e+08,1.234357e+08,0.95,512,ETSP MM-3,NaN,New Arterial Link or Interchange
3,Delta/ Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,2.259772e+07,2.553285e+07,0.25,638,NaN,NaN,Added Freeway Lanes or Major Interchange Impro...
4,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Randy Pape Belltine Highway and p...,"ODOT, Eugene",28100000.0,2030-2034,4.308515e+07,4.868131e+07,1.1,312,MM-20,Updated - Rob,Added Freeway Lanes or Major Interchange Impro...


In [22]:
df45ex.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedCost,EstimatedYearofConstruction,YearofConstructionCostMin,YearofConstructionCostMax,Length,RTP,JurisdictionalProject#,Category
0,Beltline Local Arterial Bridge,Beaver Street to Delta Highway,Construct new 2-lane arterial bridge over the ...,"ODOT, Lane County, City of Eugene",83000000.0,2025-2029,NaN,NaN,0.95,512,ETSP MM-3,New Arterial Link or Interchange
1,Eugene-Springfield Highway (SR-126),at Main Street,Construct interchange,ODOT,50000000.0,2030-2034,NaN,NaN,0.0,27,STSP R-43,New Arterial Link or Interchange
2,Eugene-Springfield Highway (SR-126),at 52nd Street,Construct interchange,ODOT,40000000.0,2025-2029,NaN,NaN,0.0,30,STSP R-40,New Arterial Link or Interchange
3,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",28100000.0,2030-2034,NaN,NaN,1.1,312,ETSP MM-20,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,20000000.0,2020-2024,NaN,NaN,0.25,638,ETSP MM-3 (part),Added Freeway Lanes or Major Interchange Impro...


In [24]:
len(df45ex.RTP.unique())

218

In [33]:
df45ex_uni = df45ex[df45ex.columns.drop(["EstimatedCost", "YearofConstructionCostMin", "YearofConstructionCostMax"])]

In [37]:
df45ex_uni.drop_duplicates(subset='RTP', keep=False, inplace=True)

<ipython-input-37-aa41aa65dd80>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df45ex_uni.drop_duplicates(subset='RTP', keep=False, inplace=True)


In [38]:
df45ex_uni.shape

(188, 9)

In [40]:
len(df45ex_uni.RTP.unique())

188

In [39]:
df45ex_uni.head()

,Name,GeographicLimits,Description,PrimaryJurisdiction,EstimatedYearofConstruction,Length,RTP,JurisdictionalProject#,Category
3,Randy Pape Beltline Highway,Roosevelt Boulevard to W. 11th Avenue,Add lanes on Belltine Highway and provide inte...,"ODOT, Eugene",2030-2034,1.1,312,ETSP MM-20,Added Freeway Lanes or Major Interchange Impro...
4,Delta/Beltline Interchange,NaN,Interim/safety improvements; replace/revise ex...,ODOT,2020-2024,0.25,638,ETSP MM-3 (part),Added Freeway Lanes or Major Interchange Impro...
6,Main Street,at 48th Street,Traffic control improvements,Springfield,2025-2029,0.0,69,NaN,Arterial Capacity Improvements
7,Main Street,at Mountaingate Drive,Traffic control improvements - signal,Springfield,2025-2029,0.0,75,R-48,Arterial Capacity Improvements
8,42nd Street,at Marcola Road,Roundabout,Springfield,2020-2024,0.0,712,R-32,Arterial Capacity Improvements


In [23]:
df45ex.shape

(254, 12)